## Schema registry

In [64]:
import requests
import json

In [65]:
## Список 
response = requests.get("http://localhost:8081/subjects")
response.json()

[]

In [54]:
### УДАЛЕНИЕ
import requests

# URL для удаления схемы
url = 'http://localhost:8081/subjects/my-avro-topic-value'

# Отправляем DELETE-запрос
response = requests.delete(url)

# Проверяем результат
if response.status_code == 200:
    print(f"Схема для 'my-topic4-value' успешно удалена.")
else:
    print(f"Ошибка при удалении схемы: {response.status_code}\n{response.text}")


Ошибка при удалении схемы: 404
{"error_code":40401,"message":"Subject 'my-avro-topic-value' not found. io.confluent.rest.exceptions.RestNotFoundException: Subject 'my-avro-topic-value' not found.\nio.confluent.rest.exceptions.RestNotFoundException: Subject 'my-avro-topic-value' not found.\n\tat io.confluent.kafka.schemaregistry.rest.exceptions.Errors.subjectNotFoundException(Errors.java:78)\n\tat io.confluent.kafka.schemaregistry.rest.resources.SubjectsResource.deleteSubject(SubjectsResource.java:278)\n\tat java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)\n\tat java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)\n\tat java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)\n\tat java.base/java.lang.reflect.Method.invoke(Method.java:566)\n\tat org.glassfish.jersey.server.model.internal.ResourceMethodInvocationHandlerFactory.lambda$static$0(ResourceMethodInvocationH

In [23]:
response = requests.get("http://localhost:8081/subjects/my-avro-topic-value/versions/latest")
response.json()

{'error_code': 40401,
 'message': "Subject 'my-avro-topic-value' not found. io.confluent.rest.exceptions.RestNotFoundException: Subject 'my-avro-topic-value' not found.\nio.confluent.rest.exceptions.RestNotFoundException: Subject 'my-avro-topic-value' not found.\n\tat io.confluent.kafka.schemaregistry.rest.exceptions.Errors.subjectNotFoundException(Errors.java:78)\n\tat io.confluent.kafka.schemaregistry.rest.resources.SubjectVersionsResource.getSchemaByVersion(SubjectVersionsResource.java:152)\n\tat java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)\n\tat java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)\n\tat java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)\n\tat java.base/java.lang.reflect.Method.invoke(Method.java:566)\n\tat org.glassfish.jersey.server.model.internal.ResourceMethodInvocationHandlerFactory.lambda$static$0(ResourceMethodInvocationHandlerFa

In [67]:
schema_id = 1  # Замените на нужный ID
url = f'http://localhost:8081/schemas/ids/{schema_id}'
response = requests.get(url)

if response.status_code == 200:
    schema = response.json().get('schema')
    print('Полученная схема:', schema)
else:
    print('Ошибка при получении схемы:', response.text)


Полученная схема: {"title":"UserList","type":"array","items":{"type":"object","properties":{"name":{"type":"string"},"age":{"type":"integer"},"email":{"type":"string"}},"required":["name","age"]}}


AVRO-схема

In [45]:
#пробуем зарегистрировать несколько разновидностей

In [2]:
##
import requests
import json

url = 'http://localhost:8081/subjects/my-topic-value/versions'
schema = {
    "schema": json.dumps({
        "type": "record",
        "name": "User",
        "fields": [
            {"name": "name", "type": "string"},
            {"name": "age", "type": "int"}
            ,
            {"name": "email", "type": "int"}#, "default": ""}
        ]
    })
}
## POST-запрос!!!
response = requests.post(url, json=schema)

if response.status_code == 200:
    schema_id = response.json().get('id')
    print('Схема зарегистрирована с ID:', schema_id)
else:
    print('Ошибка при регистрации схемы:', response.text)

Схема зарегистрирована с ID: 2


In [71]:
response = requests.get("http://localhost:8081/subjects/my-avro-topic-value/versions")
response.json()

[1]

In [72]:
response = requests.get("http://localhost:8081/subjects/my-avro-topic-value/versions/latest")
response.json()

{'subject': 'my-avro-topic-value',
 'version': 1,
 'id': 3,
 'schema': '{"type":"record","name":"User","fields":[{"name":"name","type":"string"},{"name":"favorite_number","type":"long"},{"name":"favorite_color","type":"string"}]}'}

Для AVRO - Консюмер определит версию схемы по магическим байтикам.

Для JSON надо делать ручками, например - писать версию в само соообщение

AVRO

Avro-файл — это бинарный файл, который используется для хранения данных, сериализованных в формате Avro. Он состоит из двух основных частей:

Заголовок (Header): содержит метаданные и схему данных, которая описывает формат записей в файле.
Тело (Data Blocks): содержит сами данные (записи), закодированные в соответствии с указанной схемой.

In [ ]:
{"df":1}

In [28]:
#Структура Avro-файла
# 1. Заголовок (Header)
# Каждый Avro-файл начинается с заголовка, который включает в себя:

# Магическое число (magic): первые 4 байта файла — это последовательность байт
#  0x4F 0x62 0x6A 0x01 (или строка "Obj\x01"). Это используется для идентификации файла 
# как Avro-файл.
#Метаданные (Metadata): хранит JSON-объект с информацией о схеме (например, "avro.schema") 
# и другую служебную информацию.
#Синхронизационный маркер (Sync Marker): случайная последовательность из 16 байт, которая используется для синхронизации при чтении файла блоками.

In [62]:
## работа с avro 
import fastavro

# Определение схемы
schema = {
    "type": "record",
    "name": "User",
    "fields": [
        {"name": "name", "type": "string"},
        {"name": "age", "type": "int"}
    ]
}

# Данные, которые будут сериализованы
records = [
    {"name": "Alice", "age": 25},
    {"name": "Bob", "age": 30}
]

# Запись данных в Avro-файл
with open('test.avro', 'wb') as out:
    fastavro.writer(out, schema, records)


In [63]:
import fastavro

# Чтение Avro-файла
with open('test.avro', 'rb') as f:
    reader = fastavro.reader(f)
    for record in reader:
        print(record)


{'name': 'Alice', 'age': 25}
{'name': 'Bob', 'age': 30}


In [36]:
from uuid import uuid4
str(uuid4())

'11d10a2c-1188-48bf-9540-0a866c711adb'

#### Пример в kafka_avro_producer.py 
======================================

In [5]:
### Читаем сырые байтики
from confluent_kafka import Consumer

# Конфигурация консьюмера
conf = {
    'bootstrap.servers': 'localhost:9092',
    'group.id': 'my_group111',
    'auto.offset.reset': 'earliest'  # Чтение сообщений с начала, если нет сохранённых оффсетов
}

# Создаем консьюмера
consumer = Consumer(conf)

# Подписка на топик
topic = "my-avro-topic"
consumer.subscribe([topic])

print(f"Listening to topic '{topic}'...")

# Чтение сообщений
try:
    for x in range(10):
        msg = consumer.poll(1.0)  # Ожидание сообщений

        if msg is None:
            continue

        if msg.error():
            print(f"Consumer error: {msg.error()}")
            continue

        # Получение "сырых" данных сообщения
        raw_key = msg.key()  # Байты ключа
        raw_value = msg.value()  # Байты значения

        # Вывод байтов сообщения
        print(f"Raw key (bytes): {raw_key}")
        print(f"Raw value (bytes): {raw_value}")

        # Если хотите вывести их в текстовом виде (для строк):
        print(f"Key (decoded): {raw_key.decode('utf-8') if raw_key else None}")
        print(f"Value (decoded): {raw_value.decode('utf-8') if raw_value else None}")

except KeyboardInterrupt:
    print("Consumer interrupted.")
finally:
    consumer.close()


Listening to topic 'my-avro-topic'...
Raw key (bytes): b'c3806228-478c-4f49-af3a-a68cd9b90a55'
Raw value (bytes): b'\x00\x00\x00\x00\x03\x06qwe\x02\x08red '
Key (decoded): c3806228-478c-4f49-af3a-a68cd9b90a55
Value (decoded):     qwered 
Raw key (bytes): b'd1a94380-1134-4700-9287-c955fce110e3'
Raw value (bytes): b'\x00\x00\x00\x00\x03\x06qwe\x02\x08red '
Key (decoded): d1a94380-1134-4700-9287-c955fce110e3
Value (decoded):     qwered 
Raw key (bytes): b'7b1ff7f8-1897-4c80-b32a-b13c561a6026'
Raw value (bytes): b'\x00\x00\x00\x00\x03\x06qwe\x02\x08red '
Key (decoded): 7b1ff7f8-1897-4c80-b32a-b13c561a6026
Value (decoded):     qwered 
Raw key (bytes): b'af0f93e1-d8ac-4920-a34f-d9f7c3f6c970'
Raw value (bytes): b'\x00\x00\x00\x00\x03\x06qwe\x02\x08red '
Key (decoded): af0f93e1-d8ac-4920-a34f-d9f7c3f6c970
Value (decoded):     qwered 


## Пытаемся читать обычным консюмером AVRO

In [6]:
from confluent_kafka import Consumer
from confluent_kafka.avro import AvroConsumer

# Конфигурация консьюмера
conf = {
    'bootstrap.servers': 'localhost:9092',
    'group.id': 'my_group11',
    'auto.offset.reset': 'earliest',  # Начинать чтение с самого начала, если нет смещений
    'schema.registry.url': 'http://localhost:8081',  # Schema Registry URL
}

# Создаем Avro-консьюмера
consumer = AvroConsumer(conf)

# Подписываемся на топик
consumer.subscribe(['my-avro-topic'])

# Чтение сообщений
try:
    for x in range(10):
        msg = consumer.poll(1.0)

        if msg is None:
            continue

        # Выводим полученное сообщение
        print(f"Получено сообщение: {msg.value()}")

except KeyboardInterrupt:
    pass
finally:
    consumer.close()


/tmp/ipykernel_144386/1827773791.py:13: DeprecationWarning: AvroConsumer has been deprecated. Use AvroDeserializer instead.
  consumer = AvroConsumer(conf)


SerializerError: Message deserialization failed for message at my-avro-topic [0] offset 0: message does not start with magic byte

In [7]:
response = requests.get("http://localhost:8081/subjects/my-avro-topic-value/versions/latest")
response.json()

{'subject': 'my-avro-topic-value',
 'version': 1,
 'id': 3,
 'schema': '{"type":"record","name":"User","fields":[{"name":"name","type":"string"},{"name":"favorite_number","type":"long"},{"name":"favorite_color","type":"string"}]}'}

## ДОПОЛНИТЕЛЬНЫЕ МАТЕРИАЛЫ

In [ ]:
import requests
import json

# Конфигурация Schema Registry
schema_registry_url = 'http://localhost:8081'
subject = 'my-json-topic-value'  # Название subject для вашей схемы

# JSON-схема с массивом пользователей
json_schema = {
    "title": "UserList",
    "type": "array",
    "items": {
        "type": "object",
        "properties": {
            "name": {"type": "string"},
            "age": {"type": "integer"},
            "email": {"type": "string"}
        },
        "required": ["name", "age"]
    }
}

# Подготовка данных для регистрации схемы
data = {
    "schema": json.dumps(json_schema),
    "schemaType": "JSON"  # Указываем тип схемы
}

# Запрос на регистрацию схемы
response = requests.post(
    f'{schema_registry_url}/subjects/{subject}/versions',
    headers={'Content-Type': 'application/json'},
    data=json.dumps(data)
)

# Проверка ответа
if response.status_code == 200:
    print(f"Схема успешно зарегистрирована: {response.json()}")
else:
    print(f"Ошибка регистрации схемы: {response.status_code} {response.text}")

In [ ]:
from confluent_kafka import Producer
from confluent_kafka.schema_registry import SchemaRegistryClient
from confluent_kafka.schema_registry.json_schema import JSONSerializer
from confluent_kafka.serialization import SerializationContext, MessageField
import json
import uuid

# Конфигурация Schema Registry
schema_registry_conf = {'url': "http://localhost:8081"}
schema_registry_client = SchemaRegistryClient(schema_registry_conf)

# Получаем последнюю версию схемы из Schema Registry
subject = 'my-json-topic-value'  # Субъект (subject), под которым схема зарегистрирована
schema_response = schema_registry_client.get_latest_version(subject)
schema_id = schema_response.schema_id
json_schema_str = schema_response.schema.schema_str
print(json_schema_str)

In [ ]:
# Создаем JSONSerializer с использованием загруженной схемы
json_serializer = JSONSerializer(json_schema_str, schema_registry_client)

# Конфигурация продюсера Kafka
producer_conf = {
    'bootstrap.servers': 'localhost:9092'
}
producer = Producer(producer_conf)

# Топик Kafka
topic = 'my-json-topic'

# Пример JSON-сообщения, которое соответствует схеме
data = [
    {"name": "Alice", "age": 30, "email": "alice@example.com"},
    {"name": "Bob", "age": 25, "email": "bob@example.com"}
]

# Функция для обратного вызова при доставке сообщения
def delivery_report(err, msg):
    if err is not None:
        print(f"Delivery failed for record {msg.key()}: {err}")
    else:
        print(f"Record {msg.key()} successfully produced to {msg.topic()} [{msg.partition()}] at offset {msg.offset()}")

# Производство сообщений в топик Kafka
try:
    key = str(uuid.uuid4())  # Генерация уникального ключа для каждого сообщения
    serialized_value = json_serializer(data, SerializationContext(topic, MessageField.VALUE))
    
    producer.produce(
        topic=topic,
        key=key,
        value=serialized_value,
        on_delivery=delivery_report
    )
    producer.flush()  # Дождаться отправки всех сообщений
except Exception as e:
    print(f"Error: {e}")

print("Message sent successfully.")

In [ ]:
## пишем другую схему
# Создаем JSONSerializer с использованием загруженной схемы
json_serializer = JSONSerializer(json_schema_str, schema_registry_client)

# Конфигурация продюсера Kafka
producer_conf = {
    'bootstrap.servers': 'localhost:9092'
}
producer = Producer(producer_conf)

# Топик Kafka
topic = 'my-json-topic'

# Пример JSON-сообщения, которое соответствует схеме
data = [
    {"name1": "Alice", "age": 30, "email": "alice@example.com"},
    {"name2": "Bob", "age": 25, "email": "bob@example.com"}
]

# Функция для обратного вызова при доставке сообщения
def delivery_report(err, msg):
    if err is not None:
        print(f"Delivery failed for record {msg.key()}: {err}")
    else:
        print(f"Record {msg.key()} successfully produced to {msg.topic()} [{msg.partition()}] at offset {msg.offset()}")

# Производство сообщений в топик Kafka
try:
    key = str(uuid.uuid4())  # Генерация уникального ключа для каждого сообщения
    serialized_value = json_serializer(data, SerializationContext(topic, MessageField.VALUE))
    
    producer.produce(
        topic=topic,
        key=key,
        value=serialized_value,
        on_delivery=delivery_report
    )
    producer.flush()  # Дождаться отправки всех сообщений
except Exception as e:
    print(f"Error: {e}")

print("Message sent successfully.")